This is a notebook for a simple DMRG algorithm: Modern MPS Version

When writing the julia code shown in this notebook, I refer to Mr.Min Long's python code. 

https://colab.research.google.com/github/DavidGoing/DMRG/blob/main/DMRG_MPS.ipynb

We take 1D Heisenberg model as an example, show the underlying code of MPS version DMRG.

We DO NOT use good quantum numbers in the system.

In [2]:
using Einsum
using TensorOperations
using KrylovKit
using LinearAlgebra
#initial E and F for the left and right vacuum states
function initial_E(W)
    E = zeros(1,size(W,1),1)
    E[1,1,1] = 1
    return E
end
function initial_F(W)
    F = zeros(1,size(W,3),1)
    F[1,end,1] = 1
    return F
end
# MPS-MPO-MPS contract from left
# here we use the macro: @einsum A[i, j] := B[i, k] * C[k, j]
function contract_from_left(W,A,E,B)
    @einsum tensor1[a,k,s,j] := E[i,a,k]*A[i,s,j]
    @einsum tensor2[k,j,b,t] := tensor1[a,k,s,j]*W[a,s,b,t]
    @einsum tensor3[j,b,l] := tensor2[k,j,b,t]*B[k,t,l]
    return tensor3
end
# MPS-MPO-MPS contract from right
function contract_from_right(W,A,F,B)
    @einsum tensor1[i,s,b,l] := F[j,b,l]*A[i,s,j]
    @einsum tensor2[i,a,t,l] := tensor1[i,s,b,l]*W[a,s,b,t]
    @einsum tensor3[i,a,k] := tensor2[i,a,t,l]*B[k,t,l]
    return tensor3
end
#construct F lib
# MPO is the W chain
# Alist and Blist are the MPS: the chain of A and B
# MF is the right boundary of F
function construct_F(Alist,MPO,Blist)
    MF = initial_F(MPO[end])
    F = [MF]
    for i = length(MPO):-1:2
        f = contract_from_right(MPO[i],Alist[i],F[1],Blist[i])
        F = vcat([f],F)
    end
    return F
end
#construct E lib
#ME is the left boundary of E
function construct_E(Alist,MPO,Blist)
    ME = initial_E(MPO[1])
    E = [ME]
    return E
end
# calculate expectation value of MPS
function Expectation(Alist,MPO,Blist)
    E = initial_E(MPO[1])
    for i = 1:length(MPO)
        E = contract_from_left(MPO[i],Alist[i],E,Blist[i])
    end
    return E[1,1,1]
end
# coarse-graining of two site MPO into one site
function coarse_grain_MPO(W1,W2)
    @einsum W[a,s,u,c,t,v] := W1[a,s,b,t]*W2[b,u,c,v]
    ts = size(W) #tensor size
    W12 = reshape(W,(ts[1],ts[2]*ts[3],ts[4],ts[5]*ts[6]))
    return W12
end
# product of two MPO sites
function product_W(W1,W2)
    @einsum W[a,c,s,b,d,u] := W1[a,s,b,t]*W2[c,t,d,u]
    ts = size(W) #tensor size
    W12 = reshape(W,(ts[1]*ts[2],ts[3],ts[4]*ts[5],ts[6]))
    return W12
end
# product of two Many-Body MPO 
function product_MPO(MPO1,MPO2)
    @assert length(MPO1)==length(MPO2)
    result = []
    for i = 1:length(MPO1)
        result = vcat(result,[product_W(MPO1[i],MPO2[i])])
    end
    return result
end

# coarse-graining of two-site MPS in to one site
function coarse_grain_MPS(A1,A2)
    @einsum A12[i,s,t,k] := A1[i,s,j]*A2[j,t,k]
    ts = size(A12) #tensor size
    A12 = reshape(A12,(ts[1],ts[2]*ts[3],ts[4]))
    return A12
end
#reversive process of MPS coarse graining: SVD factorize
function fine_grain_MPS(A,dims)
    @assert size(A,2) == dims[1]*dims[2]
    ts = size(A) #tensor size
    Theta = reshape(A,(ts[1],dims[1],dims[2],ts[3]))
    @tensor Theta[s,i,t,k] := Theta[i,s,t,k]
    Theta = reshape(Theta,(ts[1]*dims[1],dims[2]*ts[3]))
    F  = svd(Theta)
    bond1 = length(F.U)/dims[1]/ts[1]
    bond2 = length(F.Vt)/dims[2]/ts[3]
    bond1 = convert(Int,bond1)
    bond2 = convert(Int,bond2)
    U = reshape(F.U,(dims[1],ts[1],bond1))
    Vt = reshape(F.Vt,(bond2,dims[2],ts[3]))
    S = F.S
    return U,S,Vt
end
# U,S,V truncation
function truncate_MPS(U,S,Vt,m)
    m = min(length(S),m)
    trunc = sum(S[m+1:end])
    S = S[1:m]
    U = U[:,:,1:m]
    Vt = Vt[1:m,:,:]
    S = S/norm(S)
    return U,S,Vt,trunc,m
end
# get the ProjMPO
function HamiltonianMultiply(E,W,F,A)
    @einsum tensor1[j,s,a,k] := E[i,a,k]*A[i,s,j]
    @einsum tensor2[j,b,t,k] := tensor1[j,s,a,k]*W[a,s,b,t]
    @einsum R[k,t,l] :=tensor2[j,b,t,k]*F[j,b,l]
    return R
end

function optimize_two_sites(A1,A2,W1,W2,E,F,m,direction)
    W = coarse_grain_MPO(W1,W2)
    A0 = coarse_grain_MPS(A1,A2)
    H_eff(A) = HamiltonianMultiply(E,W,F,A)
    Eig,AA,info = eigsolve(H_eff, A0, 1, :SR)
    U,S,Vt = fine_grain_MPS(AA[1],[size(A1,2),size(A2,2)])
    U,S,Vt,trunc,m = truncate_MPS(U,S,Vt,m)
    s_diag = diagm(S)
    if direction== "right"
        @einsum A2[i,t,k] := s_diag[i,j]*Vt[j,t,k]
        @tensor U[i,s,j] := U[s,i,j]
        return Eig[1],U,A2,trunc,m
    else
        @assert direction =="left"
        @einsum A1[i,s,j] := U[s,i,m]*s_diag[m,j]
        return Eig[1],A1,Vt,trunc,m
    end
end

function two_site_dmrg(MPS,MPO,m,sweeps)
    E = construct_E(MPS,MPO,MPS)
    F = construct_F(MPS,MPO,MPS)
    deleteat!(F,1)
    for sweep = 1:(sweeps÷2)
        for i = 1:length(MPS)-2
            Energy,MPS[i],MPS[i+1],trunc,states = optimize_two_sites(MPS[i],MPS[i+1],MPO[i],MPO[i+1],E[end],F[1],m,"right")
            println("Sweep $(sweep*2)  Sites $i $(i+1) Eenrgy $Energy states $states trunc $trunc")
            e = contract_from_left(MPO[i],MPS[i],E[end],conj(MPS[i]))
            E = vcat(E,[e])
            deleteat!(F,1)
        end
        for i = length(MPS)-1:-1:2
            Energy,MPS[i],MPS[i+1],trunc,states = optimize_two_sites(MPS[i],MPS[i+1],MPO[i],MPO[i+1],E[end],F[1],m,"left")
            println("Sweep $(sweep*2+1) Sites $i $(i+1) Energy $Energy states $states trunc $trunc")
            f = contract_from_right(MPO[i+1],MPS[i+1],F[1],conj(MPS[i+1]))
            F = vcat([f],F)
            deleteat!(E,length(E))
        end
    end
    return MPS
end
            
                

two_site_dmrg (generic function with 1 method)

In [3]:
d = 2 # local bond dimenstion
N = 100 #number of sites
D = 10 #virtue bond dimenstion
InitialA1 = zeros(1,d,1)
InitialA1[1,1,1] = 1
InitialA2 = zeros(1,d,1)
InitialA2[1,2,1] = 1
#Initial State |010101010101>
MPS = repeat([InitialA1,InitialA2],convert(Int,N/2))
I = diagm([1,1])
Z = zeros(2,2)
Sz = [0.5 0;0 -0.5]
Sp = [0 0; 1 0]
Sm = [0 1; 0 0]
# define MPO element( except the element on two boundaries)
W = zeros(5,5,2,2) #[a,b,s,t]
W[1,1,:,:] = I
W[1,2,:,:] = Sz
W[1,3,:,:] = 0.5*Sp
W[1,4,:,:] = 0.5*Sm
W[2,5,:,:] = Sz
W[3,5,:,:] = Sm
W[4,5,:,:] = Sp
W[5,5,:,:] = I
@tensor W[a,s,b,t] := W[a,b,s,t]
# define the first element of MPO
Wfirst = zeros(1,5,2,2) #[a,b,s,t]
Wfirst[1,1,:,:] = I
Wfirst[1,2,:,:] = Sz
Wfirst[1,3,:,:] = 0.5*Sp
Wfirst[1,4,:,:] = 0.5*Sm
@tensor Wfirst[a,s,b,t] := Wfirst[a,b,s,t]
# define the last element of MPO
Wlast = zeros(5,1,2,2) #[a,b,s,t]
Wlast[2,1,:,:] = Sz
Wlast[3,1,:,:] = Sm
Wlast[4,1,:,:] = Sp
Wlast[5,1,:,:] = I
@tensor Wlast[a,s,b,t] := Wlast[a,b,s,t]
# construct MPO chain
MPO = vcat([Wfirst],repeat([W],N-2),[Wlast]);
HamSquared = product_MPO(MPO,MPO)
MPS = two_site_dmrg(MPS,MPO,D,8)
Energy = Expectation(MPS,MPO,conj(MPS))
println("Final energy expectation value $Energy")
H2 = Expectation(MPS,HamSquared,MPS)
println("variance = $(H2-Energy*Energy)")

Sweep 2  Sites 1 2 Eenrgy -25.059016994374947 + 0.0im states 2 trunc 0.0
Sweep 2  Sites 2 3 Eenrgy -25.18969262078592 + 0.0im states 2 trunc 0.0
Sweep 2  Sites 3 4 Eenrgy -25.395800958670193 + 0.0im states 2 trunc 0.0
Sweep 2  Sites 4 5 Eenrgy -25.560586239348073 + 0.0im states 2 trunc 0.0
Sweep 2  Sites 5 6 Eenrgy -25.743669962857084 + 0.0im states 2 trunc 0.0
Sweep 2  Sites 6 7 Eenrgy -25.917602338043913 + 0.0im states 2 trunc 0.0
Sweep 2  Sites 7 8 Eenrgy -26.095754582727768 + 0.0im states 2 trunc 0.0
Sweep 2  Sites 8 9 Eenrgy -26.271867352999635 + 0.0im states 2 trunc 0.0
Sweep 2  Sites 9 10 Eenrgy -26.44893838109645 + 0.0im states 2 trunc 0.0
Sweep 2  Sites 10 11 Eenrgy -26.625551828355015 + 0.0im states 2 trunc 0.0
Sweep 2  Sites 11 12 Eenrgy -26.802381755192936 + 0.0im states 2 trunc 0.0
Sweep 2  Sites 12 13 Eenrgy -26.97910872737826 + 0.0im states 2 trunc 0.0
Sweep 2  Sites 13 14 Eenrgy -27.15588451889701 + 0.0im states 2 trunc 0.0
Sweep 2  Sites 14 15 Eenrgy -27.33263712291239

Sweep 3 Sites 86 87 Energy -42.65417280621707 + 0.0im states 4 trunc 0.0
Sweep 3 Sites 85 86 Energy -42.66712491240823 + 0.0im states 4 trunc 0.0
Sweep 3 Sites 84 85 Energy -42.68159093796299 + 0.0im states 4 trunc 0.0
Sweep 3 Sites 83 84 Energy -42.69482552752153 + 0.0im states 4 trunc 0.0
Sweep 3 Sites 82 83 Energy -42.709020365071694 + 0.0im states 4 trunc 0.0
Sweep 3 Sites 81 82 Energy -42.72243965741455 + 0.0im states 4 trunc 0.0
Sweep 3 Sites 80 81 Energy -42.7364672654661 + 0.0im states 4 trunc 0.0
Sweep 3 Sites 79 80 Energy -42.750006059466365 + 0.0im states 4 trunc 0.0
Sweep 3 Sites 78 79 Energy -42.76392987715944 + 0.0im states 4 trunc 0.0
Sweep 3 Sites 77 78 Energy -42.77754540790159 + 0.0im states 4 trunc 0.0
Sweep 3 Sites 76 77 Energy -42.791404479141434 + 0.0im states 4 trunc 0.0
Sweep 3 Sites 75 76 Energy -42.805069026214696 + 0.0im states 4 trunc 0.0
Sweep 3 Sites 74 75 Energy -42.8188875528719 + 0.0im states 4 trunc 0.0
Sweep 3 Sites 73 74 Energy -42.83258329563422 + 0

Sweep 4  Sites 38 39 Eenrgy -43.928213950758725 + 0.0im states 8 trunc 0.0
Sweep 4  Sites 39 40 Eenrgy -43.93038130549767 + 0.0im states 8 trunc 0.0
Sweep 4  Sites 40 41 Eenrgy -43.93233702265267 + 0.0im states 8 trunc 0.0
Sweep 4  Sites 41 42 Eenrgy -43.93448335783417 + 0.0im states 8 trunc 0.0
Sweep 4  Sites 42 43 Eenrgy -43.93645535351358 + 0.0im states 8 trunc 0.0
Sweep 4  Sites 43 44 Eenrgy -43.93858457149379 + 0.0im states 8 trunc 0.0
Sweep 4  Sites 44 45 Eenrgy -43.94057015179062 + 0.0im states 8 trunc 0.0
Sweep 4  Sites 45 46 Eenrgy -43.94268541103542 + 0.0im states 8 trunc 0.0
Sweep 4  Sites 46 47 Eenrgy -43.94468230263573 + 0.0im states 8 trunc 0.0
Sweep 4  Sites 47 48 Eenrgy -43.94678616675206 + 0.0im states 8 trunc 0.0
Sweep 4  Sites 48 49 Eenrgy -43.9487924561198 + 0.0im states 8 trunc 0.0
Sweep 4  Sites 49 50 Eenrgy -43.950887012628954 + 0.0im states 8 trunc 0.0
Sweep 4  Sites 50 51 Eenrgy -43.95290109100543 + 0.0im states 8 trunc 0.0
Sweep 4  Sites 51 52 Eenrgy -43.95498

Sweep 5 Sites 55 56 Energy -44.08023330715739 + 0.0im states 10 trunc 0.0030013061495282305
Sweep 5 Sites 54 55 Energy -44.08078406002733 + 0.0im states 10 trunc 0.003669369969910012
Sweep 5 Sites 53 54 Energy -44.081085575258165 + 0.0im states 10 trunc 0.0030189651964547667
Sweep 5 Sites 52 53 Energy -44.0816243736708 + 0.0im states 10 trunc 0.003645041572639628
Sweep 5 Sites 51 52 Energy -44.08193167335448 + 0.0im states 10 trunc 0.003033436535813354
Sweep 5 Sites 50 51 Energy -44.08246051821527 + 0.0im states 10 trunc 0.0036252943109693647
Sweep 5 Sites 49 50 Energy -44.082772685399114 + 0.0im states 10 trunc 0.0030446483183228404
Sweep 5 Sites 48 49 Energy -44.083293510088346 + 0.0im states 10 trunc 0.0036102618973881573
Sweep 5 Sites 47 48 Energy -44.083609576439436 + 0.0im states 10 trunc 0.003052461711010195
Sweep 5 Sites 46 47 Energy -44.084124291045974 + 0.0im states 10 trunc 0.0036001701826816096
Sweep 5 Sites 45 46 Energy -44.08444322380294 + 0.0im states 10 trunc 0.00305666

Sweep 6  Sites 44 45 Eenrgy -44.10784694283399 + 0.0im states 10 trunc 0.011448711584377595
Sweep 6  Sites 45 46 Eenrgy -44.10795289605476 + 0.0im states 10 trunc 0.009641574100860188
Sweep 6  Sites 46 47 Eenrgy -44.10799913235773 + 0.0im states 10 trunc 0.011443277655464285
Sweep 6  Sites 47 48 Eenrgy -44.108103241090795 + 0.0im states 10 trunc 0.009653324971651818
Sweep 6  Sites 48 49 Eenrgy -44.10814912430175 + 0.0im states 10 trunc 0.011444737302725272
Sweep 6  Sites 49 50 Eenrgy -44.10825176023561 + 0.0im states 10 trunc 0.009661219383109237
Sweep 6  Sites 50 51 Eenrgy -44.10829744998613 + 0.0im states 10 trunc 0.011453732626332152
Sweep 6  Sites 51 52 Eenrgy -44.10839902524297 + 0.0im states 10 trunc 0.009665504566827006
Sweep 6  Sites 52 53 Eenrgy -44.108444689011414 + 0.0im states 10 trunc 0.011470839554239724
Sweep 6  Sites 53 54 Eenrgy -44.10854565113318 + 0.0im states 10 trunc 0.009666328722248568
Sweep 6  Sites 54 55 Eenrgy -44.1085914647967 + 0.0im states 10 trunc 0.011496

Sweep 7 Sites 63 64 Energy -44.11256105447122 + 0.0im states 10 trunc 0.011139271216328203
Sweep 7 Sites 62 63 Energy -44.11261348296793 + 0.0im states 10 trunc 0.014724061857587754
Sweep 7 Sites 61 62 Energy -44.11259821152906 + 0.0im states 10 trunc 0.011166061545401361
Sweep 7 Sites 60 61 Energy -44.1126506724316 + 0.0im states 10 trunc 0.014702485828109022
Sweep 7 Sites 59 60 Energy -44.112636671979885 + 0.0im states 10 trunc 0.01118699878572497
Sweep 7 Sites 58 59 Energy -44.11268922374567 + 0.0im states 10 trunc 0.01468483726089718
Sweep 7 Sites 57 58 Energy -44.11267632401892 + 0.0im states 10 trunc 0.011202798804064716
Sweep 7 Sites 56 57 Energy -44.112729043888294 + 0.0im states 10 trunc 0.014672331966040293
Sweep 7 Sites 55 56 Energy -44.11271708156981 + 0.0im states 10 trunc 0.011214012005755898
Sweep 7 Sites 54 55 Energy -44.11277006465125 + 0.0im states 10 trunc 0.014665839425722678
Sweep 7 Sites 53 54 Energy -44.11275888665441 + 0.0im states 10 trunc 0.011221030414157833


Sweep 8  Sites 48 49 Eenrgy -44.11435041485687 + 0.0im states 10 trunc 0.015409139035068993
Sweep 8  Sites 49 50 Eenrgy -44.11432190939185 + 0.0im states 10 trunc 0.01158967100368211
Sweep 8  Sites 50 51 Eenrgy -44.11438561780866 + 0.0im states 10 trunc 0.015408863532524375
Sweep 8  Sites 51 52 Eenrgy -44.11435842531393 + 0.0im states 10 trunc 0.011566917481290316
Sweep 8  Sites 52 53 Eenrgy -44.114424064727274 + 0.0im states 10 trunc 0.015405642548392214
Sweep 8  Sites 53 54 Eenrgy -44.114398271394606 + 0.0im states 10 trunc 0.01153527893537403
Sweep 8  Sites 54 55 Eenrgy -44.11446603331617 + 0.0im states 10 trunc 0.015399508369720188
Sweep 8  Sites 55 56 Eenrgy -44.11444171562978 + 0.0im states 10 trunc 0.01149453531404193
Sweep 8  Sites 56 57 Eenrgy -44.11451178428962 + 0.0im states 10 trunc 0.015390272640849767
Sweep 8  Sites 57 58 Eenrgy -44.11448901066588 + 0.0im states 10 trunc 0.011444195695716865
Sweep 8  Sites 58 59 Eenrgy -44.1145615630061 + 0.0im states 10 trunc 0.015377532

Sweep 9 Sites 56 57 Energy -44.11556454234144 + 0.0im states 10 trunc 0.016731595311354992
Sweep 9 Sites 55 56 Energy -44.11552373925584 + 0.0im states 10 trunc 0.011576314805865018
Sweep 9 Sites 54 55 Energy -44.11558839907775 + 0.0im states 10 trunc 0.016744525254853686
Sweep 9 Sites 53 54 Energy -44.115548523097296 + 0.0im states 10 trunc 0.011585120711264617
Sweep 9 Sites 52 53 Energy -44.11561400742969 + 0.0im states 10 trunc 0.01675455243102556
Sweep 9 Sites 51 52 Energy -44.11557498555213 + 0.0im states 10 trunc 0.011589058736714763
Sweep 9 Sites 50 51 Energy -44.11564127916165 + 0.0im states 10 trunc 0.016762103482241907
Sweep 9 Sites 49 50 Energy -44.11560302587534 + 0.0im states 10 trunc 0.011588344556809528
Sweep 9 Sites 48 49 Energy -44.11567010534493 + 0.0im states 10 trunc 0.01676728767102112
Sweep 9 Sites 47 48 Energy -44.11563252375248 + 0.0im states 10 trunc 0.011582974551206542
Sweep 9 Sites 46 47 Energy -44.11570035545597 + 0.0im states 10 trunc 0.016769902939690006
